# Questão 1 - Pedro Henrique Arruda Gonçalves

In [21]:
import pandas as pd

In [22]:
lst1 = ["Fábrica 1", "Fábrica 2", "Fábrica 3", "Demanda"]
lst2 = [800, 1100, 600, 10]
lst3 = [1300, 1400, 1200, 10]
lst4 = [400, 600, 800, 10]
lst5 = [700, 1000, 900, 10]
lst6 = [12, 17, 11, 0]

df_aux = list(zip(lst1,lst2,lst3,lst4,lst5,lst6))

df = pd.DataFrame(df_aux)

df.columns = ["", "CD1", "CD2", "CD3", "CD4", "Capacidade"]

df # Iniciando o dataframe da questão com as respectivas distâncias das fábricas
   # aos CDs, e também com as capacidades das fábricas e as demandas dos CDs 

,,CD1,CD2,CD3,CD4,Capacidade
0,Fábrica 1,800,1300,400,700,12
1,Fábrica 2,1100,1400,600,1000,17
2,Fábrica 3,600,1200,800,900,11
3,Demanda,10,10,10,10,0


In [23]:
df = df.set_index(df.iloc[0:4,0]) # Alterar o índice
df.drop("", axis=1, inplace = True) # Excluir a primeira coluna
df

,CD1,CD2,CD3,CD4,Capacidade
,,,,,
Fábrica 1,800,1300,400,700,12
Fábrica 2,1100,1400,600,1000,17
Fábrica 3,600,1200,800,900,11
Demanda,10,10,10,10,0


In [24]:
df.iloc[0:3,0] = df.iloc[0:3,0]*0.5 +100
df.iloc[0:3,1] = df.iloc[0:3,1]*0.5 +100
df.iloc[0:3,2] = df.iloc[0:3,2]*0.5 +100
df.iloc[0:3,3] = df.iloc[0:3,3]*0.5 +100

df # Descobrindo o custo total: é a distância multiplicada de 0,5 e depois somada ao um custo fixo de 100

,CD1,CD2,CD3,CD4,Capacidade
,,,,,
Fábrica 1,500.0,750.0,300.0,450.0,12
Fábrica 2,650.0,800.0,400.0,600.0,17
Fábrica 3,400.0,700.0,500.0,550.0,11
Demanda,10.0,10.0,10.0,10.0,0


In [25]:
df.Capacidade.drop("Demanda")


Fábrica 1    12
Fábrica 2    17
Fábrica 3    11
Name: Capacidade, dtype: int64

In [26]:
df_capacidades = df.Capacidade.drop("Demanda")
df_capacidades


Fábrica 1    12
Fábrica 2    17
Fábrica 3    11
Name: Capacidade, dtype: int64

In [27]:
df_demandas = df.loc["Demanda"].drop("Capacidade")
df_demandas

CD1    10.0
CD2    10.0
CD3    10.0
CD4    10.0
Name: Demanda, dtype: float64

In [28]:
df_custos = df.iloc[:-1,:-1]
df_custos

,CD1,CD2,CD3,CD4
,,,,
Fábrica 1,500.0,750.0,300.0,450.0
Fábrica 2,650.0,800.0,400.0,600.0
Fábrica 3,400.0,700.0,500.0,550.0


## Pyomo

In [29]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [30]:
modelo = pyo.ConcreteModel()

### Criando índices

In [31]:
modelo.i = Set(initialize= df_custos.index, doc='Fábricas')
modelo.j = Set(initialize= df_custos.columns, doc = "CDs")

### Criando os parâmetros

In [32]:
modelo.capacidades = Param (modelo.i, initialize= df_capacidades, doc='Capacidades')
modelo.demandas = Param(modelo.j, initialize=df_demandas, doc='Demandas')

In [33]:
dicionario_custos = df_custos.stack().to_dict()
dicionario_custos

{('Fábrica 1', 'CD1'): 500.0,
 ('Fábrica 1', 'CD2'): 750.0,
 ('Fábrica 1', 'CD3'): 300.0,
 ('Fábrica 1', 'CD4'): 450.0,
 ('Fábrica 2', 'CD1'): 650.0,
 ('Fábrica 2', 'CD2'): 800.0,
 ('Fábrica 2', 'CD3'): 400.0,
 ('Fábrica 2', 'CD4'): 600.0,
 ('Fábrica 3', 'CD1'): 400.0,
 ('Fábrica 3', 'CD2'): 700.0,
 ('Fábrica 3', 'CD3'): 500.0,
 ('Fábrica 3', 'CD4'): 550.0}

In [34]:
modelo.custos = Param(modelo.i, modelo.j, initialize = dicionario_custos, doc='Custos')

### Criando as variáveis

In [35]:
modelo.x = Var(modelo.i, modelo.j, bounds=(0.0,None), doc='Quantidades Enviadas')

### Criando as restrições

In [36]:
def regra_capacidade(modelo, i):
    return sum(modelo.x[i,j] for j in modelo.j) <= modelo.capacidades[i]

def regra_demanda(modelo, j):
    return sum(modelo.x[i,j] for i in modelo.i) >= modelo.demandas[j]


modelo.restr_capacidade = Constraint(modelo.i, rule = regra_capacidade)
modelo.restr_demanda = Constraint(modelo.j, rule = regra_demanda)

### Função Objetivo

In [37]:
def custo_total(modelo):
    return sum(modelo.custos[i,j] * modelo.x[i,j] for i in modelo.i for j in modelo.j)

modelo.Z = Objective(rule=custo_total, sense=minimize) 

In [38]:
opt = SolverFactory("gurobi")
opt.solve(modelo)

{'Problem': [{'Name': 'x13', 'Lower bound': 20200.0, 'Upper bound': 20200.0, 'Number of objectives': 1, 'Number of constraints': 8, 'Number of variables': 13, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 13, 'Number of nonzeros': 25, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.001041412353515625', 'Error rc': 0, 'Time': 0.3741133213043213}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [39]:
for v in modelo.component_data_objects(Var):
    print (str(v), v.value)
    
print("O valor ótimo encontrado foi: R$", modelo.Z())


x[Fábrica 1,CD1] 0.0
x[Fábrica 1,CD2] 0.0
x[Fábrica 1,CD3] 2.0
x[Fábrica 1,CD4] 10.0
x[Fábrica 2,CD1] 0.0
x[Fábrica 2,CD2] 9.0
x[Fábrica 2,CD3] 8.0
x[Fábrica 2,CD4] 0.0
x[Fábrica 3,CD1] 10.0
x[Fábrica 3,CD2] 1.0
x[Fábrica 3,CD3] 0.0
x[Fábrica 3,CD4] 0.0
O valor ótimo encontrado foi: R$ 20200.0
